In [1]:
!pip install transformers -qq
!pip install sentencepiece -qq
!pip install tokenizer -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 3.6 MB/s eta 0:00:00


##Load Dataset

In [2]:

import numpy as np
import pandas as pd

train = pd.read_csv('/content/train.csv')
dev = pd.read_csv('/content/dev.csv')
test = pd.read_csv('/content/test.csv')


train['free_text'] = train['free_text'].astype(str)
X_train = train['free_text']
y_train = train["label_id"]

dev['free_text'] = dev['free_text'].astype(str)
X_valid = dev['free_text']
y_valid = dev['label_id']

test['free_text'] = test['free_text'].astype(str)
X_test = test['free_text']
y_test = test['label_id']


## Set Cuda

In [3]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

torch.cuda.is_available()

True

# Extract feature by using PhoBERT

In [4]:
import pandas as pd
from glob import glob

train_sentences = list(train['free_text'].values)
train_labels = list(train['label_id'].values)

dev_sentences = list(dev['free_text'].values)
dev_labels = list(dev['label_id'].values)

test_sentences = list(test['free_text'].values)
test_labels = list(test['label_id'].values)

Load tokenizer of PhoBERT

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')

In [6]:
#choose max_length for phobert model based on the input length

max_length = 0
list_len=[]
for sentence in train_sentences:
    length = len(tokenizer.tokenize(sentence))
    list_len.append(length)

from collections import Counter
Counter(list_len).most_common(100)

[(6, 1792),
 (7, 1735),
 (5, 1639),
 (8, 1586),
 (9, 1430),
 (10, 1347),
 (4, 1307),
 (11, 1199),
 (12, 1067),
 (3, 979),
 (13, 933),
 (14, 824),
 (15, 749),
 (16, 651),
 (2, 619),
 (17, 545),
 (18, 517),
 (19, 402),
 (20, 389),
 (21, 309),
 (22, 296),
 (23, 256),
 (24, 246),
 (25, 220),
 (27, 200),
 (1, 172),
 (26, 166),
 (28, 163),
 (29, 159),
 (30, 125),
 (34, 115),
 (31, 109),
 (33, 97),
 (37, 94),
 (35, 94),
 (32, 93),
 (41, 76),
 (38, 70),
 (36, 66),
 (40, 61),
 (39, 58),
 (42, 57),
 (43, 54),
 (48, 44),
 (45, 42),
 (46, 42),
 (44, 39),
 (49, 37),
 (50, 35),
 (47, 34),
 (52, 28),
 (56, 26),
 (60, 26),
 (51, 26),
 (68, 25),
 (65, 24),
 (57, 24),
 (53, 24),
 (69, 24),
 (54, 23),
 (61, 22),
 (66, 20),
 (55, 20),
 (62, 19),
 (67, 18),
 (71, 17),
 (59, 17),
 (78, 16),
 (70, 16),
 (73, 16),
 (64, 16),
 (63, 13),
 (75, 13),
 (88, 12),
 (58, 12),
 (72, 11),
 (76, 11),
 (74, 10),
 (93, 10),
 (86, 9),
 (77, 9),
 (85, 8),
 (82, 8),
 (95, 7),
 (83, 6),
 (80, 6),
 (91, 6),
 (98, 5),
 (90, 5),

In [7]:
# Encode train label

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_labels)
encoded_labels = le.transform(train_labels)
encoded_test_labels = le.transform(dev_labels)

In [8]:
# Tokens IDs tensor

def encoder_generator(sentences,labels):

    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):

        sent_index.append(index)

        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=20,
                                             pad_to_max_length=True,
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0).cuda()
    attention_masks = torch.cat(attention_masks,dim=0).cuda()
    labels = torch.tensor(labels).cuda()
    sent_index = torch.tensor(sent_index).cuda()

    return sent_index,input_ids,attention_masks,labels

train_sent_index,train_input_ids,train_attention_masks,train_encoded_label_tensors = encoder_generator(train_sentences,encoded_labels)
dev_sent_index,dev_input_ids,dev_attention_masks,dev_encoded_label_tensors = encoder_generator(dev_sentences,encoded_test_labels)
print('Original: ', train_sentences[0])
print('Token IDs:', train_input_ids[0])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  Em được làm fan cứng luôn rồi nè ❤️ reaction quá hay quá cute coi mấy giờ này quá hợp lí =]]]
Token IDs: tensor([    0,  1350,    11,    47,  1818,  2951,   236,   182, 21091,     3,
            3,  2938, 43230,   204,   118,   204,  5109,  3915,   774,     2],
       device='cuda:0')


In [9]:
# Connvert train, dev input by using TensorDataset

from torch.utils.data import TensorDataset,random_split

train_dataset = TensorDataset(train_input_ids,train_attention_masks,train_encoded_label_tensors)
dev_dataset = TensorDataset(dev_input_ids,dev_attention_masks,dev_encoded_label_tensors)

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(dev_dataset)))

train data samples is 24048
valid data samples is 2672


In [10]:
# Set cuda by using device

from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(dev_dataset,
                              sampler=RandomSampler(dev_dataset),
                              batch_size=bs)

Load model PhoBERT

In [11]:
from transformers import AutoModel

phoBert = AutoModel.from_pretrained('vinai/phobert-base')
phoBert = phoBert.to(device)

# Build CNN

In [12]:
# Build class CNN

import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):

        super().__init__()

        self.fc_input = nn.Linear(embedding_dim,embedding_dim)

        self.conv_0 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[0])

        self.conv_1 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[1])

        self.conv_2 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[2])

        self.conv_3 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[3])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, encoded):

        #embedded = [batch size, sent len, emb dim]
        embedded = self.fc_input(encoded)
        #print(embedded.shape)

        embedded = embedded.permute(0, 2, 1)
        #print(embedded.shape)

        #embedded = [batch size, emb dim, sent len]

        conved_0 = F.relu(self.conv_0(embedded))
        conved_1 = F.relu(self.conv_1(embedded))
        conved_2 = F.relu(self.conv_2(embedded))
        conved_3 = F.relu(self.conv_3(embedded))

        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        pooled_3 = F.max_pool1d(conved_3, conved_3.shape[2]).squeeze(2)

        #pooled_n = [batch size, n_fibatlters]

        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2, pooled_3), dim = 1).cuda())

        #cat = [batch size, n_filters * len(filter_sizes)]

        result =  self.fc(cat)

        #print(result.shape)

        return result

In [13]:
# Hyperparameters

EMBEDDING_DIM = 768
N_FILTERS = 32
FILTER_SIZES = [1,2,3,5]
OUTPUT_DIM = len(le.classes_)
DROPOUT = 0.1
PAD_IDX = tokenizer.pad_token_id

cnn = CNN(EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
cnn = cnn.to(device)

In [14]:
# Optimizer and criterion

import torch.optim as optim

model_prameters = list(phoBert.parameters())+list(cnn.parameters())

optimizer = optim.Adam(model_prameters,lr=2e-5,eps=1e-8)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [15]:
# Calculate accuracy per batch during train

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).cuda()

In [16]:
# Def for training

from tqdm import tqdm

def train():

    epoch_loss = 0
    epoch_acc = 0

    phoBert.train()
    cnn.train()

    for batch in tqdm(train_data_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        optimizer.zero_grad()

        embedded = phoBert(b_input_ids,b_input_mask)[0]

        predictions = cnn(embedded)

        loss = criterion(predictions, b_labels)

        acc = categorical_accuracy(predictions, b_labels)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(train_data_loader), epoch_acc / len(train_data_loader)

In [17]:
# Class for predict label

import numpy as np

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label

In [18]:
# Evaluate loss, acc  and f1-macro

from sklearn.metrics import classification_report,accuracy_score,f1_score
def eval():
    epoch_loss = 0

    total_predictions = []
    total_true = []

    all_true_labels = []
    all_pred_labels = []

    phoBert.eval()
    cnn.eval()

    with torch.no_grad():

        for batch in tqdm(valid_data_loader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            embedded = phoBert(b_input_ids,b_input_mask)[0]
            predictions = cnn(embedded)

            loss = criterion(predictions, b_labels)
            epoch_loss += loss.item()

            predictions = predictions.detach().cpu().numpy()

            label_ids = b_labels.to('cpu').numpy()

            pred,true = predictions_labels(predictions,label_ids)

            all_pred_labels.extend(pred)
            all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')

    avg_val_loss = epoch_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))

    return avg_val_loss,avg_val_accuracy,macro_f1_score

In [19]:
# Time for training

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [20]:
# Set device and gpu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

phoBert.cuda()

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

# Training

In [21]:
epochs = 20

best_macro_f1 = float('0')

for epoch in range(epochs):

    start_time = time.time()
    train_loss,train_acc = train()
    valid_loss,valid_acc,macro_f1 = eval()
    end_time = time.time()


    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        torch.save(phoBert,'/content/phobert_cnn_model_part1_'+'task2a_2.pt')
        torch.save(cnn,'/content/phobert_cnn_model_part2_'+'task2a_2.pt')
        print("model saved")

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. acc: {valid_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. F1: {macro_f1*100:.2f}%')
    print('=============Epoch Ended==============')

100%|██████████| 21/21 [00:03<00:00,  6.56it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      0.87      0.92      2424
           1       0.00      0.00      0.00         0
           2       0.40      0.44      0.42       248

    accuracy                           0.83      2672
   macro avg       0.46      0.44      0.44      2672
weighted avg       0.91      0.83      0.87      2672

accuracy = 0.83
model saved
Epoch: 01 | Epoch Time: 1m 34s
	Train Loss: 0.545 | Train acc: 82.98%
	 Val. Loss: 0.473 |  Val. acc: 83.16%
	 Val. Loss: 0.473 |  Val. F1: 44.44%
=============Epoch Ended==============


100%|██████████| 21/21 [00:03<00:00,  6.96it/s]


              precision    recall  f1-score   support

           0       0.97      0.88      0.93      2405
           1       0.11      0.62      0.18        37
           2       0.48      0.57      0.52       230

    accuracy                           0.85      2672
   macro avg       0.52      0.69      0.54      2672
weighted avg       0.92      0.85      0.88      2672

accuracy = 0.85
model saved
Epoch: 02 | Epoch Time: 1m 31s
	Train Loss: 0.424 | Train acc: 84.86%
	 Val. Loss: 0.415 |  Val. acc: 85.37%
	 Val. Loss: 0.415 |  Val. F1: 54.37%
=============Epoch Ended==============


100%|██████████| 21/21 [00:02<00:00,  7.04it/s]


...

              precision    recall  f1-score   support

           0       0.94      0.92      0.93      2237
           1       0.39      0.50      0.44       163
           2       0.54      0.54      0.54       272

    accuracy                           0.86      2672
   macro avg       0.62      0.65      0.64      2672
weighted avg       0.86      0.86      0.86      2672

accuracy = 0.86
Epoch: 18 | Epoch Time: 1m 31s
	Train Loss: 0.039 | Train acc: 98.62%
	 Val. Loss: 0.712 |  Val. acc: 85.52%
	 Val. Loss: 0.712 |  Val. F1: 63.51%
=============Epoch Ended==============


100%|██████████| 21/21 [00:02<00:00,  7.05it/s]


              precision    recall  f1-score   support

           0       0.95      0.91      0.93      2270
           1       0.37      0.56      0.45       142
           2       0.55      0.57      0.56       260

    accuracy                           0.86      2672
   macro avg       0.62      0.68      0.64      2672
weighted avg       0.88      0.86      0.87      2672

accuracy = 0.86
Epoch: 19 | Epoch Time: 1m 31s
	Train Loss: 0.041 | Train acc: 98.62%
	 Val. Loss: 0.749 |  Val. acc: 86.04%
	 Val. Loss: 0.749 |  Val. F1: 64.47%
=============Epoch Ended==============


100%|██████████| 21/21 [00:02<00:00,  7.04it/s]

              precision    recall  f1-score   support

           0       0.94      0.92      0.93      2248
           1       0.42      0.47      0.44       190
           2       0.49      0.56      0.52       234

    accuracy                           0.85      2672
   macro avg       0.62      0.65      0.63      2672
weighted avg       0.86      0.85      0.86      2672

accuracy = 0.85
Epoch: 20 | Epoch Time: 1m 31s
	Train Loss: 0.038 | Train acc: 98.66%
	 Val. Loss: 0.765 |  Val. acc: 85.29%
	 Val. Loss: 0.765 |  Val. F1: 63.13%
=============Epoch Ended==============


In [22]:
# Save PhoBERT and CNN

torch.save(phoBert,'module2_part1.pt')
torch.save(cnn,'module2_part2.pt')

## EVALUATING

In [23]:
# Load phobert and cnn

import torch
phoBert = torch.load(r'/content/phobert_cnn_model_part1_task2a_2.pt')
cnn = torch.load(r'/content/phobert_cnn_model_part2_task2a_2.pt')
phoBert.eval()
cnn.eval()


CNN(
  (fc_input): Linear(in_features=768, out_features=768, bias=True)
  (conv_0): Conv1d(768, 32, kernel_size=(1,), stride=(1,))
  (conv_1): Conv1d(768, 32, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(768, 32, kernel_size=(3,), stride=(1,))
  (conv_3): Conv1d(768, 32, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=128, out_features=3, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

Predict label from true label

In [24]:
test_sent_index, test_input_ids, test_attention_masks, test_encoded_label_tensors = encoder_generator(test_sentences,test_labels)
test_dataset = TensorDataset(test_input_ids,test_attention_masks,test_encoded_label_tensors)

test_data_loader = DataLoader(test_dataset,
                              sampler=RandomSampler(test_dataset),
                              batch_size=bs)

all_pred_labels = []
all_true_labels = []

with torch.no_grad():
  for batch in tqdm(test_data_loader):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    embedded = phoBert(b_input_ids,b_input_mask)[0]
    predictions = cnn(embedded)


    predictions = predictions.detach().cpu().numpy()

    label_ids = b_labels.to('cpu').numpy()

    pred, true = predictions_labels(predictions, label_ids)

    all_pred_labels.extend(pred)
    all_true_labels.extend(true)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 53/53 [00:07<00:00,  7.18it/s]


In [25]:
# The final score in the test set (classification report)

print(classification_report(all_pred_labels,all_true_labels, digits = 4))

              precision    recall  f1-score   support

           0     0.9284    0.9261    0.9273      5562
           1     0.4189    0.3932    0.4057       473
           2     0.5218    0.5566    0.5386       645

    accuracy                         0.8527      6680
   macro avg     0.6231    0.6253    0.6239      6680
weighted avg     0.8531    0.8527    0.8528      6680



In [26]:
# Confusion matrix in thetest set

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(all_true_labels, all_pred_labels)
cm

array([[5151,  192,  205],
       [ 177,  186,   81],
       [ 234,   95,  359]])